### Load libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import os
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
from xgboost import XGBClassifier
import shap

# Skopt functions
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# Set path to search waveform tables
os.chdir("../../Data/")

### Function to one-hot encode the Clinical DF, and define the new variable names

In [2]:
def one_hot_encoding_DF(df):

    one_hot = OneHotEncoder()
    # Woman Education
    encoded = one_hot.fit_transform(df[['womanEducation']])
    df[['no/primary', 'secondary/technology', 'university']] = encoded.toarray()
    # Woman Work
    encoded = one_hot.fit_transform(df[['womanWork']])
    df[['unemployed', 'autonomous', 'private/student/other']] = encoded.toarray()
    # Habits
    encoded = one_hot.fit_transform(df[['habits']])
    df[['habits_no', 'habits_stop_pregnancy', 'habits_yes']] = encoded.toarray()
    # Dropping Unnecessary Columns
    df = df.drop(columns=['womanEducation', 'womanWork', 'habits'])
    # Re-arrange columsn to place outcomes at the end
    cols = df.columns.tolist()
    cols = cols[:18] + cols[24:] + cols[18:24]
    df = df[cols]

    return df

### Load IMPACT – Clinical Data + Fetal biometrics + EchoPI

SGA encoded using the BW percentile calculated using the local standard (Figueras et al.)

In [3]:
# CLINICAL DATA (Demographics + Previous/Current pregnancy + Mother Measurements)
dfClinical = pd.read_csv('jointDF28WeeksIMPACT_SGA_Intergrowth-21.csv', index_col='ID')
dfClinical.drop('dataset', axis=1, inplace=True)

# Replacing NaNs in binary variables with 0 – Assumption of normality
dfClinical[['highBloodPressure', 'convulsions', 'GDM', 'anaemiaOrIron', 'bleeding']] = dfClinical[[
    'highBloodPressure', 'convulsions', 'GDM', 'anaemiaOrIron', 'bleeding']].fillna(0)

# Drop samples missing continuous variables & outcomes
dfClinical = dfClinical[dfClinical.isna().any(axis=1) == False]

# One hot encoding categorical variables
dfClinical = one_hot_encoding_DF(dfClinical)
dfClinical.index = [int(idx[6:]) for idx in dfClinical.index]

# FETAL BIOMETRICS
dfBiometrics = pd.read_excel('IMPACT/IMPACT_ecocardio_raw.xlsx', index_col='Cod')
dfBiometrics = dfBiometrics[['PC_ecoIMPACT','DBP_ecoIMPACT','LF_ecoIMPACT','PA_ecoIMPACT','EG_ecoIMPACT','EFW_Hadlock','EGparto_cod']]
dfBiometrics.columns = ['HC', 'BPD', 'FL', 'AC', 'GA', 'EFW','GA_birth']
dfBiometrics = dfBiometrics[dfBiometrics.isna().any(axis=1) == False]

# ECHO PI
dfEchoPI = pd.read_excel('IMPACT/IMPACT_ecocardio_raw.xlsx', index_col='Cod')
dfEchoPI = dfEchoPI[['UA_ecoIMPACT','ACM_ecoIMPACT','CPR_calc']]
dfEchoPI.columns = ['UA PI', 'MCA PI', 'CPR']
dfEchoPI = dfEchoPI[dfEchoPI.isna().any(axis=1) == False]

# KEEP ONLY COMMON INDICES
commonIndices = [idx for idx in dfClinical.index if idx in dfBiometrics.index and idx in dfEchoPI.index]
dfBiometrics = dfBiometrics.loc[commonIndices].round(decimals=2)
dfEchoPI = dfEchoPI.loc[commonIndices].round(decimals=2)
dfClinical = dfClinical.loc[commonIndices].round(decimals=2)

dfImpact = pd.concat([dfClinical, dfBiometrics,dfEchoPI ], axis=1)

### Load FEDOC – Clinical Data + Fetal biometrics + EchoPI

In [5]:
# CLINICAL DATA (Demographics + Previous/Current pregnancy + Mother Measurements)
dfClinical = pd.read_csv('jointDF28WeeksFEDOC_SGA_Intergrowth-21.csv', index_col='ID')
dfClinical.drop('dataset', axis=1, inplace=True)
# Replacing NaNs in binary variables with 0 – Assumption of normality
dfClinical[['highBloodPressure', 'convulsions', 'GDM', 'anaemiaOrIron', 'bleeding']] = dfClinical[[
    'highBloodPressure', 'convulsions', 'GDM', 'anaemiaOrIron', 'bleeding']].fillna(0)


dfGA_birth = pd.read_excel('FEDOC/Pilot Fedoc - full data file March 2022.xlsx', index_col='ML Study ID')
dfGA_birth = dfGA_birth[['Gestational age in weeks']]
dfGA_birth.columns = ['GA_birth']

# Replacing NaNs in percentile with 0 – Not to drop death cases
#dfClinical['Weight_percentile'] = dfClinical['Weight_percentile'].fillna(0)

# Drop samples missing continuous variables & outcomes
#dfClinical = dfClinical[dfClinical.isna().any(axis=1) == False]

# One hot encoding
dfClinical = one_hot_encoding_DF(dfClinical)
dfClinical.index = [int(idx[5:]) for idx in dfClinical.index]

# FETAL BIOMETRICS
dfBiometrics = pd.read_csv('FEDOC/Form 3 - with biometry z score.csv', index_col='f3_ml_sid')
# If studies are duplicated, keep only the FU visit (higher GA)
idx = dfBiometrics.index
to_keep = np.logical_not(idx.duplicated(keep='last'))
dfBiometrics = dfBiometrics.loc[to_keep]
dfBiometrics = dfBiometrics[['HC', 'BPD', 'FL', 'AC','GA_weeks','EFW_Hadlock']]
dfBiometrics.columns = ['HC', 'BPD', 'FL', 'AC', 'GA', 'EFW']
dfBiometrics = dfBiometrics[dfBiometrics.isna().any(axis=1) == False]

# Echo PI
dfEchoPI = pd.read_excel('FEDOC/EchoPIs - z score.xlsx', index_col='Study ID')
# If studies are duplicated, keep only the FU visit (higher GA)
dfEchoPI.index = [str(idx)[0:4] for idx in dfEchoPI.index]
to_keep = np.logical_not(dfEchoPI.index.duplicated(keep='last'))
dfEchoPI = dfEchoPI.loc[to_keep]
dfEchoPI.index = [int(idx) for idx in dfEchoPI.index]
dfEchoPI = dfEchoPI[['UA PI', 'MCA PI', 'CPR']]
dfEchoPI = dfEchoPI[dfEchoPI.isna().any(axis=1) == False]

# KEEP ONLY COMMON INDICES
commonIndices = [idx for idx in dfClinical.index if idx in dfBiometrics.index and idx in dfEchoPI.index]

dfBiometrics = dfBiometrics.loc[commonIndices].round(decimals=2)
dfEchoPI = dfEchoPI.loc[commonIndices].round(decimals=2)
dfClinical = dfClinical.loc[commonIndices].round(decimals=2)
dfClinical = dfClinical.loc[commonIndices].round(decimals=2)
dfGA_birth = dfGA_birth.loc[commonIndices].round(decimals=2)

dfFedoc = pd.concat([dfClinical, dfBiometrics, dfEchoPI, dfGA_birth], axis=1)

In [ ]:
# set the max columns to none
pd.set_option('display.max_columns', None)
dfFedoc[dfFedoc['neonatalDeath']==1]
dfFedoc.isna().sum()
dfFedoc.shape
dfFedoc.describe()
dfFedoc[dfFedoc['preterm'].isnull()].index.tolist() # Only 2 cases missing partum data – these are excluded

### Input variables to be used in the different (incremental) experiments 

In [ ]:
input_features = {'Set1': ['womanAge', 'previouslyPregnant', 'previousPreterm', 'previousFetalDeath', 'antenatalCare', 'highBloodPressure', 'convulsions', 'GDM', 'anaemiaOrIron', 'feverOrAntibiotics', 'bleeding',
                  'maternalWeight', 'maternalHeight', 'systolicBP', 'diastolicBP', 'hemoglobin', 'GA_meas', 'GA_US', 'no/primary', 'secondary/technology', 'university', 'unemployed', 'autonomous', 'private/student/other',
                  'habits_no', 'habits_stop_pregnancy', 'habits_yes'], 
                  
                  'Set2': ['HC', 'BPD', 'FL', 'AC', 'GA'],

                  'Set3': ['UA PI', 'MCA PI', 'CPR', 'GA'], 

                  'Set4': ['womanAge', 'previouslyPregnant', 'previousPreterm', 'previousFetalDeath', 'antenatalCare', 'highBloodPressure', 'convulsions', 'GDM', 'anaemiaOrIron', 'feverOrAntibiotics', 'bleeding',
                  'maternalWeight', 'maternalHeight', 'systolicBP', 'diastolicBP', 'hemoglobin', 'GA_meas', 'GA_US', 'no/primary', 'secondary/technology', 'university', 'unemployed', 'autonomous', 'private/student/other',
                  'habits_no', 'habits_stop_pregnancy', 'habits_yes', 'HC', 'BPD', 'FL', 'AC'], 

                  'Set5': ['womanAge', 'previouslyPregnant', 'previousPreterm', 'previousFetalDeath', 'antenatalCare', 'highBloodPressure', 'convulsions', 'GDM', 'anaemiaOrIron', 'feverOrAntibiotics', 'bleeding',
                  'maternalWeight', 'maternalHeight', 'systolicBP', 'diastolicBP', 'hemoglobin', 'GA_meas', 'GA_US', 'no/primary', 'secondary/technology', 'university', 'unemployed', 'autonomous', 'private/student/other',
                  'habits_no', 'habits_stop_pregnancy', 'habits_yes', 'UA PI', 'MCA PI', 'CPR'],

                  'Set6': ['HC', 'BPD', 'FL', 'AC', 'GA', 'UA PI', 'MCA PI', 'CPR'], 

                  'Set7': ['womanAge', 'previouslyPregnant', 'previousPreterm', 'previousFetalDeath', 'antenatalCare', 'highBloodPressure', 'convulsions', 'GDM', 'anaemiaOrIron', 'feverOrAntibiotics', 'bleeding',
                  'maternalWeight', 'maternalHeight', 'systolicBP', 'diastolicBP', 'hemoglobin', 'GA_meas', 'GA_US', 'no/primary', 'secondary/technology', 'university', 'unemployed', 'autonomous', 'private/student/other',
                  'habits_no', 'habits_stop_pregnancy', 'habits_yes', 'HC', 'BPD', 'FL', 'AC', 'UA PI', 'MCA PI', 'CPR']
                  }

### Data exploration

Variable inspection – Cohorts comparison (normalized histograms = probabilities)


In [ ]:
'''for feat in input_features['Set1']:

    tmp = dfImpact[feat]
    bin_width = (max(tmp) - min(tmp)) / 10
    if bin_width == 0: bin_width = .1
    weights = np.ones_like(tmp)/float(len(tmp))
    plt.hist(tmp, weights = weights, alpha=0.5, label='IMPACT', bins = np.arange(min(tmp), max(tmp) + bin_width, bin_width))
    
    tmp = dfFedoc[feat]
    weights = np.ones_like(tmp)/float(len(tmp))
    plt.hist(tmp, weights = weights, alpha=0.5, label='FEDOC', bins = np.arange(min(tmp), max(tmp) + bin_width, bin_width))
    plt.legend(loc='upper right')
    plt.title(feat)
    plt.xlabel('Value')
    plt.ylabel('Probability')
    plt.show()'''

Variable inspection – Class distribution (normalized histograms = probabilities)

In [ ]:
'''Outcome = 'SGA'

for feat in input_features['Set4']:

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle(feat)

    tmp = dfImpact.loc[dfImpact[Outcome]==0,feat]
    bin_width = (max(tmp) - min(tmp)) / 10
    if bin_width == 0: bin_width = .1
    weights = np.ones_like(tmp)/float(len(tmp))
    ax1.hist(tmp, weights = weights, bins = np.arange(min(tmp), max(tmp) + bin_width, bin_width),  alpha=0.5, label='Normal')
    tmp = dfImpact.loc[dfImpact[Outcome]==1,feat]
    weights = np.ones_like(tmp)/float(len(tmp))
    ax1.hist(tmp, weights = weights, bins = np.arange(min(tmp), max(tmp) + bin_width, bin_width), alpha=0.5, label=Outcome)
    ax1.legend(loc='upper right')
    ax1.set_title('IMPACT')
    ax1.set_xlabel('Value')
    ax1.set_ylabel('Probability')

    tmp = dfFedoc.loc[dfFedoc[Outcome]==0,feat]
    bin_width = (max(tmp) - min(tmp)) / 10
    if bin_width == 0: bin_width = .1
    weights = np.ones_like(tmp)/float(len(tmp))
    ax2.hist(tmp, weights = weights, bins = np.arange(min(tmp), max(tmp) + bin_width, bin_width), alpha=0.5, label='Normal')
    tmp = dfFedoc.loc[dfFedoc[Outcome]==1,feat]
    weights = np.ones_like(tmp)/float(len(tmp))
    ax2.hist(tmp, weights = weights, bins = np.arange(min(tmp), max(tmp) + bin_width, bin_width), alpha=0.5, label=Outcome)
    ax2.legend(loc='upper right')
    ax2.set_title('FEDOC')
    ax2.set_xlabel('Value')
    ax2.set_ylabel('Probability')

    plt.show()'''

### Outcome variable to be used – remove missing values

In [ ]:
# List of Outcomes --> ['SGA', 'weightPercentile', 'cSection', 'neonatalDeath', 'birthWeight', 'preterm']

Outcome = 'SGA'

#Outcome = 'preterm < 36'
#dfImpact[Outcome] = np.array(dfImpact['GA_birth'] < 36).astype(int)
#dfFedoc[Outcome] = np.array(dfFedoc['GA_birth'] < 36).astype(int)

#dfImpact['Low_BW'] = (dfImpact['birthWeight'] <= 2.5) * 1
#dfFedoc['Low_BW'] = (dfFedoc['birthWeight'] <= 2.5) * 1
#Outcome = 'Low_BW'

dfImpact = dfImpact.dropna(subset=[Outcome])
dfFedoc = dfFedoc.dropna(subset=[Outcome])

print( (sum(dfImpact[Outcome])/len(dfImpact))*100)
print( (sum(dfFedoc[Outcome])/len(dfFedoc))*100)

### Dictionary of experiments

In [ ]:
expDict = {'Exp1': ['X1_train','y1_train','X1_test','y1_test'], #Impact–Impact 
           'Exp2': ['X2_train','y2_train','X2_test','y2_test'], #Fedoc–Fedoc 
           'Exp3': ['X1_train','y1_train','X2_test','y2_test'], #Impact–Fedoc
           'Exp4': ['X2_train','y2_train','X1_test','y1_test'], #Fedoc–Impact
           }

#expDict = {'Exp2': ['X2_train','y2_train','X2_test','y2_test']} #Fedoc–Fedoc

### 3.1-A Grid Search and Cross Validation

Rather than the naive approach followed before, we now implement a cross-validation technique to find the model hyperparameters ('max_depth', 'n_estimators', 'learning_rate') that maximize the classification score (AUC). 

In [ ]:
def gridSearch_CV(X_train, y_train):
    
    w = (1 - np.mean(y_train))/np.mean(y_train)
    estimator = XGBClassifier(scale_pos_weight = w, eval_metric = 'logloss', verbosity = 0, silent=True, nthread=4, seed=42)

    parameters = {
        'max_depth': range (1, 10, 1),
        'n_estimators': range(50, 1001, 50),
        'learning_rate': np.arange(0.1, 0.51, 0.1)
    }

    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=parameters,
        scoring = 'roc_auc',
        n_jobs = -1, # uses all available processors
        cv = 5, # make (stratified) k-fold CV; using the StratifiedKFold method
        verbose=True
    )

    grid_search.fit(X_train, y_train)

    return grid_search

### 3.1-B Bayesian (hyperparameter) optimization and Cross Validation

Rather than the brute force (grid search), which optimizes the true objective function, we now implement a Bayesian search, which optimizes a proxy function (surrogate function). Thus, we do exploration rather than (brute force) exploitation. With the drawback of finding a local (rather than a global) minima for the obtective function, this method favours computational efficiency, and allows us to search over a much larger space of hyperparameters. 

In [ ]:
def BayesSearch_CV(X_train, y_train):
    
    w = (1 - np.mean(y_train))/np.mean(y_train)
    estimator = XGBClassifier(scale_pos_weight = w, eval_metric = 'logloss', verbosity = 0, silent=True, seed=42)

    # Setting the search space
    search_spaces = {'learning_rate': Real(.1, .50, 'uniform'),
                    'max_depth': Integer(2, 10),
                    'subsample': Real(0.1, 1.0, 'uniform'),
                    'colsample_bytree': Real(0.1, 1.0, 'uniform'), # subsample ratio of columns by tree
                    'reg_lambda': Real(1e-9, 100., 'uniform'), # L2 regularization
                    'reg_alpha': Real(1e-9, 100., 'uniform'), # L1 regularization
                    'n_estimators': Integer(50, 500)}

    bayesian = BayesSearchCV(
        estimator=estimator,
        search_spaces=search_spaces,
        scoring = 'roc_auc',
        n_iter = 25,
        n_jobs = -1, # uses all available processors
        cv = 5, # make (stratified) k-fold CV; using the StratifiedKFold method
        random_state=42,
        verbose=False
    )

    bayesian.fit(X_train, y_train)

    return bayesian

### 3.2 (OPTIONAL) XGBoost Classifier – Feature selection

In [ ]:
# feature selection
def select_features(X_train, y_train, X_test, i):
    # configure to select a subset of features
    fs = SelectFromModel(XGBClassifier(n_estimators=i), max_features=10)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    feature_names = fs.get_feature_names_out(input_features)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs, feature_names

In [ ]:
# Set at 10% FPR – normally used in related bibliography

def getThresholdFromROC(y_true, y_pred, fpr_desired = .1):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred,)
    return np.interp(fpr_desired, fpr, thresholds)

### Evaluate the classification model on test data

In [ ]:
def model_evaluation(model,X_test,y_test):
    
    yhat_test_prob = model.predict_proba(X_test)

    th = getThresholdFromROC(y_test,yhat_test_prob[:,1])
    yhat_test = yhat_test_prob[:,1] > th

    # ROC AUC
    AUC =  roc_auc_score(y_test, yhat_test_prob[:,1])*100

    # Confusion Matrix 
    CM = confusion_matrix(y_test,yhat_test)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]

    # Print Classification scores
    Sn = (TP/(TP+FN))*100
    Sp = (TN/(TN+FP))*100
    PPV = (TP/(TP+FP))*100
    NPV = (TN/(TN+FN))*100

    train_AUC = model.best_score_*100
    #print('Grid search parameters: ', model.best_params_)

    return train_AUC, AUC, Sn, Sp, PPV, NPV

## Launch classification models

1. For all defined feature sets
2. Make 10 iterations (to average classification scores)
3. Launch all scenarios defined in expDict. For each scenario:
    
    - Grid search (depth, n_estimators, learning rate) + 5-fold cross validation
    - Bayesian optimization (preferred over Grid search)
    - (OPTIONAL) Keep the 10 most informative features
<br/><br/>
    
4. Print SHAP-values of the best performing model
5. Stats of classification scores among nIts

In [ ]:
nIts = 10
nSets = input_features.keys()
nExps = expDict.keys()
values = ['train_AUC', 'AUC', 'Sn', 'Sp', 'PPV', 'NPV','bestModel']

# Create results dictionary
results = {}
for set in nSets:
    results[set] = {}
    for iter in range(nIts):
        results[set][iter] = {}
        for exp in nExps:
            results[set][iter][exp] = {}
            for val in values:
                results[set][iter][exp][val] = None


for set in nSets:

    print(set)

    X1 = dfImpact[input_features[set]]
    y1 = dfImpact[Outcome]
    #y1 = np.array(dfImpact[Outcome] == 1).astype(int) & np.array(dfImpact['cSection'] == 0).astype(int) 
    X2 = dfFedoc[input_features[set]]
    y2 = dfFedoc[Outcome]
    #y2 = np.array(dfFedoc[Outcome] == 1).astype(int) & np.array(dfFedoc['cSection'] == 0).astype(int)

    for iter in tqdm(range(nIts)):

        X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = .3, random_state=iter, stratify = y1)
        X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = .3, random_state=iter, stratify = y2)

        for exp in nExps: 

            X_train = globals()[expDict[exp][0]].astype(float)
            y_train = globals()[expDict[exp][1]].astype(float)
            X_test = globals()[expDict[exp][2]].astype(float)
            y_test = globals()[expDict[exp][3]].astype(float)

            # In the test set, predict only in the group from 31 to 33 weeks (max. overlap for GA_US in both cohorts)
            #to_include = X_test['GA_US'].between(31,33,inclusive=True)
            #X_test = X_test[to_include]
            #y_test = y_test[to_include]

            # Grid search & 5-fold CV to find the best performing model, while preventing overfitting
            #best_model = gridSearch_CV(X_train,y_train)
            best_model = BayesSearch_CV(X_train,y_train)
            # Model evaluation on test data
            train_AUC, AUC, Sn, Sp, PPV, NPV = model_evaluation(best_model,X_test,y_test)

            results[set][iter][exp]['train_AUC'] = np.round(train_AUC,1)
            results[set][iter][exp]['AUC'] = np.round(AUC,1)
            results[set][iter][exp]['Sn'] = np.round(Sn,1)
            results[set][iter][exp]['Sp'] = np.round(Sp,1)
            results[set][iter][exp]['PPV'] = np.round(PPV,1)
            results[set][iter][exp]['NPV'] = np.round(NPV,1)
            results[set][iter][exp]['bestModel'] = best_model


### Print ROC curve

In [ ]:
def print_ROC(model,X_test,y_test,set,exp):

    yhat_test_prob = model.predict_proba(X_test)

    fpr, tpr, _ = roc_curve(y_test,  yhat_test_prob[:,1])

    # Create ROC curve
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=fpr ,y=tpr, 
                            name = ('Classifier (AUC = %.2f)' % (round(roc_auc_score(y_test, yhat_test_prob[:,1])*100))),
                            line=dict(color='royalblue', width=4, dash='solid')))
    fig.add_trace(go.Scatter(x=[0,1],y=[0,1], 
                            name = 'Random Chance', 
                            line=dict(color='black', width=4, dash='dot')))

    fig.update_layout(
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate", 
        height=600,
        width=600, 
        legend=dict(yanchor="top", y=0.98, xanchor="left", x=0.02))

    #fig.show()

    # Save figure
    fig.write_image('/Users/sergio/Desktop/' + set + exp + '_ROC.pdf')

### Print SHAP Values

In [ ]:
def print_SHAP(model,X_test,set,exp):

    # Fits the explainer
    explainer = shap.Explainer(model.predict, X_test)
    # Calculates the SHAP values - It takes some time
    shap_values = explainer(X_test)
    shap.plots.beeswarm(shap_values,show=False)
    plt.savefig('/Users/sergio/Desktop/' + set + exp + '_SHAP.png',bbox_inches ='tight')
    plt.close()

### Print results (best performing model)

In [ ]:
for set in nSets:

    X1 = dfImpact[input_features[set]]
    y1 = dfImpact[Outcome]
    X2 = dfFedoc[input_features[set]]
    y2 = dfFedoc[Outcome]

    for exp in nExps:

        listAUC = ([results[set][iter][exp]['AUC'] for iter in range(nIts)])
        iter = listAUC.index(max(listAUC))

        X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = .3, random_state=iter, stratify = y1)
        X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = .3, random_state=iter, stratify = y2)

        model = results[set][iter][exp]['bestModel']
        X_test = globals()[expDict[exp][2]].astype(float)
        y_test = globals()[expDict[exp][3]].astype(float)

        print_ROC(model,X_test,y_test,set,exp)
        print_SHAP(model,X_test,set,exp)


### Save results

Extended results are saved in json format. The model object is deleted before saving numeric accuracy results (AUC_train, AUC, Sn, Sp, PPV, NPV)

In [ ]:
for set in nSets:
    for iter in range(nIts):
        for exp in nExps: 
            del results[set][iter][exp]['bestModel']

import json

with open('/Users/sergio/Desktop/results.json', 'w') as fp:
    json.dump(results, fp)

Mean & STD across iterations for each set of features and experiment

In [ ]:
print('AUC_train')
for exp in nExps: 
    for set in nSets:
        AUC_train = ([results[set][iter][exp]['train_AUC'] for iter in range(nIts)])
        print(str(np.around(np.mean(AUC_train),decimals=1)) + u" \u00B1 " + str(np.around(np.std(AUC_train),decimals=1)))

print('\nAUC_test')
for exp in nExps: 
    for set in nSets:
        AUC = ([results[set][iter][exp]['AUC'] for iter in range(nIts)])
        print(str(np.around(np.mean(AUC),decimals=1)) + u" \u00B1 " + str(np.around(np.std(AUC),decimals=1)))

print('\nSensitivity')
for exp in nExps: 
    for set in nSets:        
        Sn = ([results[set][iter][exp]['Sn'] for iter in range(nIts)])
        print(str(np.around(np.mean(Sn),decimals=1)) + u" \u00B1 " + str(np.around(np.std(Sn),decimals=1)))

print('\nSpecificity')
for exp in nExps: 
    for set in nSets:
        Sp = ([results[set][iter][exp]['Sp'] for iter in range(nIts)])
        print(str(np.around(np.mean(Sp),decimals=1)) + u" \u00B1 " + str(np.around(np.std(Sp),decimals=1)))

print('\nPPV')
for exp in nExps: 
    for set in nSets:
        PPV = ([results[set][iter][exp]['PPV'] for iter in range(nIts)])
        print(str(np.around(np.mean(PPV),decimals=1)) + u" \u00B1 " + str(np.around(np.std(PPV),decimals=1)))

print('\nNPV')
for exp in nExps: 
    for set in nSets:
        NPV = ([results[set][iter][exp]['NPV'] for iter in range(nIts)])
        print(str(np.around(np.mean(NPV),decimals=1)) + u" \u00B1 " + str(np.around(np.std(NPV),decimals=1)))
        
